**Cài đặt thư viện**


In [ ]:
!pip install dash
!pip install neurokit2
!pip install PyWavelets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import random
import dash
from dash import dcc, html
import plotly.graph_objs as go
import base64
import io
from scipy.signal import find_peaks
import neurokit2 as nk  # Thư viện Neurokit2 để tìm đỉnh PQRST
import pywt  # Thư viện PyWavelets để lọc wavelet
from scipy.signal import butter, filtfilt, iirnotch


**Load dataset để train model**

In [ ]:
file_path = 'MIT-BIH Arrhythmia Database.csv'
data_df = pd.read_csv(file_path)

**Lấy các đặc trưng trong dataset để trainning**

In [ ]:
x_data = data_df.iloc[:, 2:13]
y_label = data_df[['type']]

**Gán nhãn cho các loại nhịp tim**

In [ ]:
y_label.replace(['N'], 'Normal', inplace=True)
y_label.replace(['VEB'], 'Supraventricular ectopic beat', inplace=True)
y_label.replace(['SVEB'], 'Ventricular ectopic beat', inplace=True)
y_label.replace(['F'], 'Fusion beat', inplace=True)
y_label.replace(['Q'], 'Unknown beat', inplace=True)

<ipython-input-32-f4902967a3fb>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-32-f4902967a3fb>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-32-f4902967a3fb>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-32-f4902967a3fb>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

**Chuyển nhãn về dạng số**

In [ ]:
encoder = LabelEncoder()
y_label = encoder.fit_transform(y_label)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



**Chia dữ liệu thành tập train và tập test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_label,test_size=0.2, random_state=101)

**Chuẩn hóa dữ liệu**

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
X_test_scaled = min_max_scaler.transform(X_test)

**Training model**

In [ ]:
model = RandomForestClassifier(random_state=101, n_estimators=150)
model.fit(X_train_scaled, y_train)

# training accuracy
print('Accuracy for the train data', model.score(X_train_scaled, y_train))

Accuracy for the train data 1.0


**Tính toán độ chính xác của model với dữ liệu test đã chia**

In [ ]:
y_pred = model.predict(X_test_scaled)
score = accuracy_score(y_test, y_pred)
print(score)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print("*** Confusion Matrix ***")
print(metrics.confusion_matrix(y_test, y_pred))

0.9833151256331314
Accuracy: 0.9833151256331314
*** Confusion Matrix ***
[[  102    38    13     0     0]
 [    3 18030    24     0    15]
 [    7    91  1243     0     4]
 [    0     4     0     0     0]
 [    0   127    10     0   427]]


**Khởi tạo app xử lý tín hiệu và đưa ra chuẩn đoán bệnh tim**

In [ ]:
# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__)
app.title = "ECG Signal with Abnormal Detection"
# Hàm lọc Wavelet Denoising
def wavelet_denoise(signal, wavelet='db6', level=4):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    coeffs[1:] = [pywt.threshold(c, value=np.std(c) / 3, mode='soft') for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)

# Hàm bộ lọc thông thấp
def lowpass_filter(signal, fs, cutoff=40, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, signal)

# Hàm bộ lọc thông cao
def highpass_filter(signal, fs, cutoff=0.5, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return filtfilt(b, a, signal)

# Hàm bộ lọc Notch
def notch_filter(signal, fs, freq=50, quality=30):
    nyquist = 0.5 * fs
    notch_freq = freq / nyquist
    b, a = iirnotch(notch_freq, quality)
    return filtfilt(b, a, signal)
# Hàm tính nhịp tim (BPM)
def calculate_bpm(peaks, fs):
    peak_times = peaks / fs
    rr_intervals = np.diff(peak_times)
    avg_rr_interval = np.mean(rr_intervals)
    heart_rate = 60 / avg_rr_interval
    return heart_rate

# Hàm tìm đỉnh R bằng phương pháp find_peaks
def find_r_peaks(ecg_signal, fs):
    distance = fs / 2.5  # Khoảng cách giữa các đỉnh R tối thiểu
    height = np.max(ecg_signal) * 0.4  # Độ cao tối thiểu của đỉnh R
    peaks, _ = find_peaks(ecg_signal, distance=distance, height=height)
    return peaks

# Hàm chỉ phát hiện PQRST (giữ nguyên phần đỉnh còn lại)
def detect_pqrst(signal, fs):
    # Phát hiện đỉnh R bằng find_peaks
    r_peaks = find_r_peaks(signal, fs)

    # Phân tích các đặc điểm PQRST bằng neurokit2
    ecg_report, waves = nk.ecg_delineate(signal, r_peaks, sampling_rate=fs, method="dwt")

    # Lấy các vị trí của P, Q, R, S, T
    p_peaks = np.array(waves.get("ECG_P_Peaks", []))
    q_peaks = np.array(waves.get("ECG_Q_Peaks", []))
    s_peaks = np.array(waves.get("ECG_S_Peaks", []))
    t_peaks = np.array(waves.get("ECG_T_Peaks", []))

    # Loại bỏ giá trị NaN và chuyển sang số nguyên
    p_peaks = p_peaks[~np.isnan(p_peaks)].astype(int)
    q_peaks = q_peaks[~np.isnan(q_peaks)].astype(int)
    s_peaks = s_peaks[~np.isnan(s_peaks)].astype(int)
    t_peaks = t_peaks[~np.isnan(t_peaks)].astype(int)

    return r_peaks, p_peaks, q_peaks, s_peaks, t_peaks
    # Hàm tính ST duration và QRS duration
def calculate_durations(r_peaks, p_peaks, q_peaks, s_peaks, t_peaks, fs):
    qrs_intervals = []
    pq_intervals = []
    qt_intervals = []
    st_intervals = []
    for i in range(min(len(r_peaks), len(q_peaks), len(s_peaks), len(t_peaks), len(p_peaks))):
      q_onset = q_peaks[i]
      s_offset = s_peaks[i]
      t_onset = t_peaks[i]
      p_onset = p_peaks[i]
      # Tính QRS duration (từ Q đến S)
      qrs_duration = (s_offset - q_onset) / fs  # Đơn vị: giây
      qrs_intervals.append(qrs_duration)
      # Tính PQ duration (từ P đến Q)
      pq_duration = (q_onset - p_onset) / fs  # Đơn vị: giây
      pq_intervals.append(pq_duration)
      # Tính QT duration (từ Q đến T)
      qt_duration = (t_onset - q_onset) / fs  # Đơn vị: giây
      qt_intervals.append(qt_duration)
      # Tính ST duration (từ S đến T)
      st_duration = (t_onset - s_offset) / fs  # Đơn vị: giây
      st_intervals.append(st_duration)
    return qrs_intervals, pq_intervals, qt_intervals, st_intervals
# Callback xử lý khi tải lên tệp và phát hiện PQRST
@app.callback(
    [dash.Output('ecg-graph', 'figure'),
     dash.Output('bpm-output', 'children'),
     dash.Output('pred-out', 'children'),
     dash.Output('alert', 'children')],  # Thêm đầu ra cảnh báo
    [dash.Input('upload-data', 'contents')],
    [dash.State('upload-data', 'filename')]
)
def update_graph(contents, filename):
    if contents is None:
        return {}, "BPM: --", "Prediction: --", ""  # Thêm giá trị trống cho cảnh báo

    # Giải mã dữ liệu đã tải lên
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')), header=None)
    ecg_signal = df[0].values
    fs = 370  # Tần số lấy mẫu

    # Áp dụng các bộ lọc
    filtered_signal = highpass_filter(ecg_signal, fs)
    filtered_signal = lowpass_filter(filtered_signal, fs)
    filtered_signal = notch_filter(filtered_signal, fs)

    # Áp dụng lọc wavelet
    clean_signal = nk.ecg_clean(filtered_signal, sampling_rate=fs, method="biosppy")

    # Phát hiện các đỉnh PQRST bằng Neurokit2 và find_peaks
    r_peaks, p_peaks, q_peaks, s_peaks, t_peaks = detect_pqrst(clean_signal, fs)

    # Prediction logic
    rr_intervals = np.diff(r_peaks) / fs
    pre_rr = [0] + rr_intervals.tolist()
    post_rr = rr_intervals.tolist() + [0]
    qrs_intervals, pq_intervals, qt_intervals, st_intervals = calculate_durations(
        r_peaks, p_peaks, q_peaks, s_peaks, t_peaks, fs
    )
    min_length = min(
        len(r_peaks), len(q_peaks), len(s_peaks), len(t_peaks), len(p_peaks),
        len(pre_rr), len(post_rr), len(qrs_intervals), len(pq_intervals),
        len(qt_intervals), len(st_intervals)
    )
    x_pred = np.array([
        pre_rr[:min_length], post_rr[:min_length], p_peaks[:min_length],
        t_peaks[:min_length], r_peaks[:min_length], s_peaks[:min_length],
        q_peaks[:min_length], qrs_intervals[:min_length], pq_intervals[:min_length],
        qt_intervals[:min_length], st_intervals[:min_length]
    ]).T
    x_pred = min_max_scaler.transform(x_pred)
    y_pred = model.predict(x_pred)

    # Tính nhịp tim (BPM)
    bpm = calculate_bpm(r_peaks, fs)

    # Tạo đồ thị với PQRST
    figure = {
        'data': [
            go.Scatter(x=np.arange(len(clean_signal)) / fs, y=clean_signal, mode='lines', name='Clean ECG Signal'),
            go.Scatter(x=r_peaks / fs, y=clean_signal[r_peaks], mode='markers', name='R Peaks', marker=dict(color='red', size=10)),
            go.Scatter(x=p_peaks / fs, y=clean_signal[p_peaks], mode='markers', name='P Peaks', marker=dict(color='blue', size=8)),
            go.Scatter(x=q_peaks / fs, y=clean_signal[q_peaks], mode='markers', name='Q Peaks', marker=dict(color='green', size=8)),
            go.Scatter(x=s_peaks / fs, y=clean_signal[s_peaks], mode='markers', name='S Peaks', marker=dict(color='orange', size=8)),
            go.Scatter(x=t_peaks / fs, y=clean_signal[t_peaks], mode='markers', name='T Peaks', marker=dict(color='purple', size=8))
        ],
        'layout': go.Layout(
            title='ECG Signal with PQRST Detection and Wavelet Filter',
            xaxis=dict(title='Time (s)'),
            yaxis=dict(title='Amplitude'),
            showlegend=True
        )
    }

    # Cảnh báo nếu BPM vượt ngưỡng
    alert = ""
    if bpm > 90:
        alert = "Cảnh báo: Nhịp tim cao bất thường!"
    elif bpm < 40:
        alert = "Cảnh báo: Nhịp tim thấp bất thường!"

    bpm_text = f"BPM: {bpm:.2f}"
    y_pred_text = f"Prediction: {encoder.inverse_transform(y_pred)[0]}"
    return figure, bpm_text, y_pred_text, alert

**Thiết lập giao diện người dùng**

In [ ]:
#Giao diện người dùng
app.layout = html.Div([
    html.H1("ECG Signal with Abnormal Detection"),

    dcc.Upload(
        id='upload-data',
        children=html.Div(['Drag and Drop or ', html.A('Select a CSV File')]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),

    html.Div(id='bpm-output', style={'fontSize': 24, 'marginTop': '20px'}),
    html.Div(id='pred-out', style={'fontSize': 24, 'marginTop': '20px'}),
    html.Div(id='alert', style={'fontSize': 24, 'color': 'red', 'marginTop': '20px'}),  # Hiển thị cảnh báo

    dcc.Loading(
        id="loading",
        type="circle",
        children=dcc.Graph(id='ecg-graph')
    ),
])

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>